In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import joblib, json, sys, os
from IPython.display import Image, display_png

sys.path.append("../script/")
from MergeBrain import WholeBrainCells
import matplotlib.pyplot as plt
import pandas as pd

import sys, os, os.path, re, csv, math
import linecache
import numpy as  np
import pandas as pd
from matplotlib import pyplot as plt
from statistics import mean,stdev
import seaborn as sns

import csv
import pprint
import ants
import tifffile
from multiprocessing import Pool
import datetime
from itertools import product

In [2]:
import numpy as np
import tifffile
import os
import pickle
import joblib
import pandas as pd
from matplotlib import pyplot as plt
import concurrent.futures
import gc
import json

dt_local = np.dtype([
    ('local_x', 'f4'), ('local_y', 'f4'), ('local_z', 'f4'),
    ('structureness', 'f4'), ('blobness', 'f4'), ('intensity', 'f4'),
    ('size', 'u2'), ('padding', 'u2'), ('intensity2', 'f4'), ('intensity3', 'f4')
])


dt_annotated = np.dtype([
    ('stitched_x', 'f4'), ('stitched_y', 'f4'), ('stitched_z', 'f4'),
    ('mapped_x', 'f4'), ('mapped_y', 'f4'), ('mapped_z', 'f4'),
    ('atlas_id', 'u2'), ('is_positive','f4'),('is_positive2','f4'),('is_positive3','f4')
])



dt_annotated_init = np.dtype([
    ('local_x', 'f4'), ('local_y', 'f4'), ('local_z', 'f4'),
    ('stitched_x', 'f4'), ('stitched_y', 'f4'), ('stitched_z', 'f4'),
    ('mapped_x', 'f4'), ('mapped_y', 'f4'), ('mapped_z', 'f4'),
    ('atlas_id', 'u2'), ('is_positive','f4'),('intensity','f4'),('structureness','f4'), # ここのis_positiveはinitial classifyによるもの
    ('intensity_2','f4'), ('intensity_3','f4')
])

dt_annotated_classified_intensity = np.dtype([
    ('local_x', 'f4'), ('local_y', 'f4'), ('local_z', 'f4'),
    ('stitched_x', 'f4'), ('stitched_y', 'f4'), ('stitched_z', 'f4'),
    ('mapped_x', 'f4'), ('mapped_y', 'f4'), ('mapped_z', 'f4'),
    ('atlas_id', 'u2'), ('is_positive','f4'), ('intensity','f4'),('structureness','f4'),
    ('intensity2','f4'),('intensity3','f4'), # ここのis_positiveはinitial classifyによるものにする
    ('norm_intensity_2','f4'), ('norm_intensity_3','f4')
])

# 必要なdtypeの定義
dt_needed = np.dtype([
    ('mapped_x', 'f4'), ('mapped_y', 'f4'), ('mapped_z', 'f4'),
    ('intensity','f4'),('structureness','f4')
])

dt_classified = np.dtype([
    ('is_positive','bool'), ('is_positive2','bool'),('is_positive3','bool')
])

dt_classified2 = np.dtype([
   ('is_positive','bool'), ('intensity','f4'),('intensity_2','f4'), ('intensity_3','f4')
])

dt_annotation2 = np.dtype([
  ('stitched_x', 'f4'), ('stitched_y', 'f4'), ('stitched_z', 'f4'),
  ('mapped_x', 'f4'), ('mapped_y', 'f4'), ('mapped_z', 'f4'),
  ('SCA_x', 'f4'), ('SCA_y', 'f4'), ('SCA_z', 'f4'),
  ('atlas_id', 'u2')
])

def process_file_all (fw_dir_nu_, file_name):
    
    try:
        
        
        fw_dir_intensities_ = fw_dir_nu_.replace("nu_R_/result", "intensities_")
        tmp_points_intensities_ = np.fromfile(os.path.join(fw_dir_intensities_, file_name), dtype=dt_annotated_classified_intensity)
        
        
        #print(len( tmp_points_intensities_))
        tmp_src = np.zeros((len(tmp_points_intensities_),), dtype=dt_annotation2)
        
        fw_dir_nu_R_ = fw_dir_nu_ #.replace("intensities_","nu_R_/result")
        tmp_points_nu_R_ = np.fromfile(os.path.join(fw_dir_nu_R_, file_name), dtype=dt_annotated_init)
        
        mask = (tmp_points_nu_R_["stitched_x"] != 0.0) & (tmp_points_nu_R_["stitched_y"] != 0.0) & (tmp_points_nu_R_["is_positive"] > 0.0)
        #mask =  data_local_nucl["atlas_id"] > 0.0
        tmp_points_nu_R_2 = tmp_points_nu_R_[mask]
        
        

        #fw_dir_nu_candidate_nu_ = fw_dir_nu_.replace("nu_R_/result","candidate_nu_")
        #tmp_points_candidate_nu_ = np.fromfile(os.path.join(fw_dir_nu_candidate_nu_, file_name), dtype=dt_local)
        #tmp_points_candidate_nu_2 = tmp_points_candidate_nu_[mask]

        #print(tmp_points_intensities_[0:1] )
        #print(tmp_points_nu_R_2[0:1])
        #print(len(tmp_points_candidate_nu_2))
        tmp_src ["stitched_x"] = tmp_points_nu_R_2 ["stitched_x"]
        tmp_src ["stitched_y"] = tmp_points_nu_R_2 ["stitched_y"]
        tmp_src ["stitched_z"] = tmp_points_nu_R_2 ["stitched_z"]
        tmp_src ["mapped_x"] = tmp_points_nu_R_2 ["mapped_x"]
        tmp_src ["mapped_y"] = tmp_points_nu_R_2 ["mapped_y"]
        tmp_src ["mapped_z"] = tmp_points_nu_R_2 ["mapped_z"]
        
        #print(tmp_src)

        #tmp_src ["atlas_id"] = tmp_points_nu_R_2 ["atlas_id"] # 0 のままにしておく

        #print(len(tmp_src))
        del  tmp_points_intensities_, tmp_points_nu_R_,  tmp_points_nu_R_2
        gc.collect()
        
    except Exception as e:
        print("emp")
        return np.empty((0, len(dt_annotation2.names)))
    
    return tmp_src

# HDoGやnu Rをやりなおすまえにintensitiesしていることがあるので、これではダメ。
#def process_file_all (fw_dir_nu_, file_name):
#    return np.fromfile(os.path.join(fw_dir_nu_, file_name), dtype=dt_annotated_classified_intensity)

In [ ]:
conditions = {
    "WT_1m": ["#4_APPmodel_Ctr1m_1_2022_1104_1550","#4_APPmodel_Ctr1m_2_2022_1108_959","#4_APPmodel_Ctr1m_3_2022_1209_1751"],
    "WT_2m": ["8w_B6JWT_atlas_1_2022_0924_1847","8w_B6JWT_atlas_2_2022_0926_1326", "8w_B6JWT_atlas_3_2022_1005_1110", "8w_B6JWT_atlas_4_2022_1010_2030","8w_B6JWT_atlas_5_2023_0423_1604",],
    "WT_3m": ["#5_APPmodel_Ctr3m_1_2023_0113_1230", "#5_APPmodel_Ctr3m_2_2023_0210_1142", "#5_APPmodel_Ctr3m_3_2023_0227_1605", "#4_APPmodel_Ctr3m_4_2023_0427_1444",],
    "WT_5m": ["#4_APPmodel_Ctr5m_1_2022_1026_252", "#4_APPmodel_Ctr5m_2_2022_1028_2336", "#4_APPmodel_Ctr5m_3_2022_1101_140", "#4_APPmodel_Ctr5m_4_2023_0512_2057"],
    "WT_7m": [ "#5_APPmodel_Ctr7m_1_2023_0206_1534","#5_APPmodel_Ctr7m_2_2023_0313_1111","#5_APPmodel_Ctr7m_3_2023_0331_2139","#4_APPmodel_Ctr7m_4_2023_0516_1559","#5_APPmodel_Ctr7m_5_2023_0522_2218"],
    "WT_9m": [ "#5_APPmodel_Ctr9m_1_2023_0130_1607", "#5_APPmodel_Ctr9m_2_2023_0216_2052" ,"#5_APPmodel_Ctr9m_3_2023_0322_2014","#5_APPmodel_Ctr9m_4_2023_0518_1449"],
    "WT_12m": ["56w_B6JWT_atlas_1_2022_1128_1548", "56w_B6JWT_atlas_2_2022_1221_1537", "56w_B6JWT_atlas_3_2022_1224_1718"],
    
    "APP_1m": ["#4_APPmodel_APP1m_1_2022_1102_1304", "#4_APPmodel_APP1m_2_2022_1106_1708","#4_APPmodel_APP1m_3_2022_1213_2151","#4_APPmodel_APP1m_4_2023_0417_2052", ],
    "APP_3m": [ "#5_APPmodel_APP3m_2_2023_0123_1645", "#5_APPmodel_APP3m_3_2023_0212_2217","#5_APPmodel_APP3m_4_2023_0214_2027","#4_APPmodel_APP3m_5_2023_0502_1423",],
   "APP_5m": ["#4_APPmodel_APP5m_1_2022_1024_1336", "#4_APPmodel_APP5m_2_2022_1027_1333","#4_APPmodel_APP5m_3_2022_1030_1257", "#4_APPmodel_APP5m_4_2023_0509_1021"],
    "APP_7m": ["#5_APPmodel_APP7m_2_2023_0315_1540","#5_APPmodel_APP7m_3_2023_0310_1401","#5_APPmodel_APP7m_4_2023_0317_1630", "#5_APPmodel_APP7m_5_2023_0520_1723"],
    "APP_9m": ["#5_APPmodel_APP9m_1_2023_0117_1352","#5_APPmodel_APP9m_2_2023_0327_1425", "#5_APPmodel_APP9m_3_2023_0329_1611", "#5_APPmodel_APP9m_4_2023_0403_1443" ],
       
    "VCP_mut_2m": ["VCPmodel_E524A_0.5x_1_2022_0624_1303", "VCPmodel_E524A_0.5x_2_2022_0626_1338", "VCPmodel_E524A_0.5x_4_2022_0922_1217", "VCPmodel_E524A_0.5x_6_2022_0628_1157"], 
   "VCP_WT_2m": ["VCPmodel_WT_0.5x_1_2022_0625_1104","VCPmodel_WT_0.5x_2_2022_0917_1808", "VCPmodel_WT_0.5x_3_2022_0627_1257", "VCPmodel_WT_0.5x_4_2022_0921_950","VCPmodel_WT_0.5x_5_2022_0923_1454","VCPmodel_WT_0.5x_6_2022_0929_953"],
    "newVCP_mut_2m": ["#5_VCPmodel_E524A_0.5x_1_2023_0120_1929","#5_VCPmodel_E524A_0.5x_2_2023_0407_1031", "#5_VCPmodel_E524A_0.5x_3_2023_0413_2036"], 
   "newVCP_WT_2m": ["#5_VCPmodel_WT_0.5x_1_2023_0405_1108","#5_VCPmodel_WT_0.5x_2_2023_0115_1852","#5_VCPmodel_WT_0.5x_3_2023_0410_1246"], 
    
    "TMT_2m": ["#3_TMTmodel_TMT3.0_8w_1_2022_1014_1410", "#3_TMTmodel_TMT3.0_8w_2_2022_1020_938", "#3_TMTmodel_TMT3.0_8w_3_2022_1023_501", "#3_TMTmodel_TMT_8w_5_2022_1205_1147",],
  "Saline_2m": ["#3_TMTmodel_Saline_8w_1_2022_1016_1318", "#3_TMTmodel_Saline_8w_2_2022_1018_2103","#3_TMTmodel_Saline_8w_3_2022_1021_1910", "#3_TMTmodel_Saline_8w_4_2022_1202_1349", "#3_TMTmodel_Saline_8w_5_2022_1217_1723", "#3_TMTmodel_Saline_8w_6_2022_1216_001"],
    
}

In [4]:
grid_mode=0
Samples = ["Neuronomics/" + path for paths in conditions.values() for path in paths]
Samples

['Neuronomics/#3_1w_B6JWT_atlas_1_2022_1103_2305',
 'Neuronomics/1w_B6JWT_atlas_2_2022_1219_1829',
 'Neuronomics/1w_B6JWT_atlas_3_2022_1223_1719',
 'Neuronomics/1w_B6JWT_atlas_4_2023_0105_1708',
 'Neuronomics/3w_B6JWT_atlas_1_2022_0925_1526',
 'Neuronomics/3w_B6JWT_atlas_2_2022_1004_928',
 'Neuronomics/3w_B6JWT_atlas_3_2022_1017_2126',
 'Neuronomics/3w_B6JWT_atlas_4_2022_1130_1051',
 'Neuronomics/#4_APPmodel_Ctr1m_1_2022_1104_1550',
 'Neuronomics/#4_APPmodel_Ctr1m_2_2022_1108_959',
 'Neuronomics/#4_APPmodel_Ctr1m_3_2022_1209_1751',
 'Neuronomics/8w_B6JWT_atlas_1_2022_0924_1847',
 'Neuronomics/8w_B6JWT_atlas_2_2022_0926_1326',
 'Neuronomics/8w_B6JWT_atlas_3_2022_1005_1110',
 'Neuronomics/8w_B6JWT_atlas_4_2022_1010_2030',
 'Neuronomics/8w_B6JWT_atlas_5_2023_0423_1604',
 'Neuronomics/#5_APPmodel_Ctr3m_1_2023_0113_1230',
 'Neuronomics/#5_APPmodel_Ctr3m_2_2023_0210_1142',
 'Neuronomics/#5_APPmodel_Ctr3m_3_2023_0227_1605',
 'Neuronomics/#4_APPmodel_Ctr3m_4_2023_0427_1444',
 'Neuronomics/#4_A

In [15]:
# classifyされていないやつでやる
cou3 = 0
#for Sample in Samples:
for Sample in Samples:#[::-1][46:]:
    
    # stitching後 & ANTs後のcellome情報を読み出し
    
    start_time = datetime.datetime.now()
    start_time2 = datetime.datetime.now()
    print("Start Start time:", start_time)
    
    try:
        if grid_mode==0:
            print("### Read parameter file ###")

            param_path = "/home/mitani/CUBIC-informatics/param/" + Sample + "/param_multichannel-rank.json"

            print(param_path)

            with open(param_path) as f:
                param = json.load(f)

            fw_dir_nu = param["dst_basedir"].replace("intensities_", "nu_R_/result") #.replace("intensities_","intensities_For_PDF_clustering")
            #fw_dir_nu = "/data4/ds1_Imaging3/Axial/Neurorology/8w_B6JWT_atlas_5_2023_0423_1604/intensities_3color_but_high_thresh/" #param["dst_basedir"].replace("intensities_","intensities_3color_but_high_thresh")
            #fw_dir_nu = "/data4/ds1_Imaging2/Axial/Neurorology/#4_APPmodel_Ctr5m_3_2022_1101_140/intensities_3color_but_high_thresh/" 
            #fw_dir_nu = "/data4/ds1_Imaging3/Axial/Neurorology/8w_B6JWT_atlas_5_2023_0423_1604/nu_R_/result/"
            print(fw_dir_nu)

            points_nu = []
            points_nu_RV = []
            tmp_points_nu = []
            tmp = []

            # sortを入れて、file名の数字が若い順にする。
            # 各ファイルの点群の数を格納する辞書を初期化
            points_length_dict_FW = {}

            print("FW reading")
            fw_dir_nu_ = fw_dir_nu + "/FW/"
            file_list = os.listdir(fw_dir_nu_)
            file_list.sort()  # ファイルリストを先にソートしてから使う
            with concurrent.futures.ProcessPoolExecutor(max_workers=5) as executor:
                points_nu = list(executor.map(process_file_all, [fw_dir_nu_]*len(file_list), file_list))

            # points_nuの各要素の長さと、対応するファイル名を辞書に保存
            for filename, points in zip(file_list, points_nu):
                points_length_dict_FW[filename] = len(points)
            #print(points_length_dict_FW)

            end_time = datetime.datetime.now()
            print("FW reading end:", end_time)
            print("Duration:", end_time - start_time)
            start_time = datetime.datetime.now()

            print("RV reading")

            points_length_dict_RV = {}

            fw_dir_nu_ = fw_dir_nu + "/RV/"
            file_list = os.listdir(fw_dir_nu_)
            file_list.sort()  # ファイルリストを先にソートしてから使う
            with concurrent.futures.ProcessPoolExecutor(max_workers=5) as executor:
                points_nu +=list(executor.map(process_file_all, [fw_dir_nu_]*len(file_list), file_list))
                points_nu_RV = list(executor.map(process_file_all, [fw_dir_nu_]*len(file_list), file_list))
            print ("READ OK")

            # points_nuの各要素の長さと、対応するファイル名を辞書に保存
            for filename, points in zip(file_list, points_nu_RV):
                points_length_dict_RV[filename] = len(points)
            #print(points_length_dict_RV)

            end_time = datetime.datetime.now()
            print("RV reading end:", end_time)
            print("Duration:", end_time - start_time)
            start_time = datetime.datetime.now()

            from concurrent.futures import ProcessPoolExecutor
            import concurrent.futures

            new_array = []

            def filter_func(arr):
                return arr if len(arr) > 0 else None

            def parallel_filter(points_nu):
                with ProcessPoolExecutor() as executor:
                    filtered = list(executor.map(filter_func, points_nu))
                return [arr for arr in filtered if arr is not None]

            #filtered_points = parallel_filter(points_nu)
            #new_array = np.concatenate(filtered_points)

            # 各配列の部分を処理する関数
            def process_array_part(arr, start_index):
                if len(arr) > 0:
                    return start_index, start_index + len(arr), arr
                else:
                    return start_index, start_index, None


            ##ここで修正した。
            fw_dir_nu = param["dst_basedir"]#.replace("nu_R_/result", "intensities_") 

            # 合計サイズの計算
            total_size = sum(len(arr) for arr in points_nu if len(arr) > 0)

            # 空の配列を作成
            new_array = np.empty((total_size,), dtype=dt_annotation2)

            # 並列処理
            with concurrent.futures.ProcessPoolExecutor() as executor:
                # 開始インデックスを計算
                start_index = 0
                future_to_part = {}
                for arr in points_nu:
                    future = executor.submit(process_array_part, arr, start_index)
                    future_to_part[future] = (start_index, len(arr))
                    start_index += len(arr)

                # 結果を収集し、配列に代入
                for future in concurrent.futures.as_completed(future_to_part):
                    start, end, part = future.result()
                    if part is not None:
                        new_array[start:end] = part

            # del points_nu_filtered, points_nu
            del points_nu, points_nu_RV, future_to_part #filtered_points
            gc.collect()

            end_time = datetime.datetime.now()
            print("parallel_filter end:", end_time)
            print("Duration:", end_time - start_time)
            start_time = datetime.datetime.now()

            print(len(new_array))
            print(new_array)



            dt_registration = np.dtype([
                ('z', 'f4'), ('y', 'f4'), ('x', 'f4')
            ]) # 50 um scale

            grid_points_df = []
            if Sample == "Neuronomics/8w_B6JWT_atlas_1_2022_0924_1847":
                print("8w-1 atlas")
                grid_points_df =  np.zeros((len(new_array),), dtype=dt_registration)
                grid_points_df["z"]  =  new_array["stitched_z"] * (1/50)
                grid_points_df["y"]  = (new_array["stitched_y"] - 17*80) * (1/50)
                grid_points_df["x"]  = (new_array["stitched_x"] - 40*80) * (1/50)
            if Sample != "Neuronomics/8w_B6JWT_atlas_1_2022_0924_1847":
                print("Not 8w-1 atlas")
                grid_points_df =  np.zeros((len(new_array),), dtype=dt_registration)
                grid_points_df["z"]  = new_array["mapped_z"] * (1/50)
                grid_points_df["y"]  = new_array["mapped_y"] * (1/50)
                grid_points_df["x"]  = new_array["mapped_x"] * (1/50)

            transformed_points = []

            transformed_points =  pd.DataFrame(transformed_points)
            grid_points_df =  pd.DataFrame(grid_points_df)

            start_time = datetime.datetime.now()
            print("Trasform Start time:", start_time)

            transformed_points = ants.apply_transforms_to_points(
                dim=3,
                points=grid_points_df,
                transformlist=["/export2/Imaging/ds4_Data4/atlas_R_test/slice_0_88_inv.mat"],
                whichtoinvert = [True]
            )

            end_time = datetime.datetime.now()
            print("Trasform End time:", end_time)
            print("Duration:", end_time - start_time)

            transformed_points = pd.DataFrame({'id': range(len(transformed_points)), 'x': transformed_points["x"], 'y': transformed_points["y"], 'z': transformed_points["z"]})

        if grid_mode==1:
            dt_stitched_corrected = np.dtype([
                ('z', 'f4'), ('y', 'f4'), ('x', 'f4'), ('id', 'u4')  # 'u4'はnp.uint32に対応
            ])
            dt_stitched2 = np.dtype([
                ('z', 'f4'), ('y', 'f4'), ('x', 'f4')
            ])
            i = cou3
            print(i)
            transformed_points = []
            grid_points_df = []
            fw_dir_nu = "/export2/Imaging/ds4_Data4/atlas_R_test/"
            #transformed_points = np.fromfile("/export2/Imaging/ds4_Data4/atlas_R_test/CUBIC-space-grid-10um-resolution_modified_transformed_" +str(i)+".bin", dtype=dt_stitched_corrected)
            #grid_points_df = np.fromfile("/export2/Imaging/ds4_Data4/atlas_R_test/CUBIC-space-grid-10um-resolution_modified_" +str(i)+".bin", dtype=dt_stitched2)
            transformed_points = np.fromfile("/export3/Imaging/Axial/Neurorology/8w_B6JWT_atlas_1_2022_0924_1847/CUBIC-space-grid-25um-resolution_transformed.bin", dtype=dt_stitched_corrected)
            grid_points_df = np.fromfile("/export3/Imaging/Axial/Neurorology/8w_B6JWT_atlas_1_2022_0924_1847/CUBIC-space-grid-25um-resolution.bin", dtype=dt_stitched2)
            transformed_points =  pd.DataFrame(transformed_points)
            grid_points_df =  pd.DataFrame(grid_points_df)
            end_time = datetime.datetime.now()
            print("Trasform End time:", end_time)
            print("Duration:", end_time - start_time)
            #dt_annotation2 = np.dtype([
            #  ('stitched_x', 'f4'), ('stitched_y', 'f4'), ('stitched_z', 'f4'),
            #  ('mapped_x', 'f4'), ('mapped_y', 'f4'), ('mapped_z', 'f4'),
            #  ('SCA_x', 'f4'), ('SCA_y', 'f4'), ('SCA_z', 'f4'),
            #  ('atlas_id', 'u2')
            #])
            new_array = np.empty((len(grid_points_df),), dtype=dt_annotation2)
            new_array ['stitched_x'] =  grid_points_df["x"]
            new_array ['stitched_y'] =  grid_points_df["y"]
            new_array ['stitched_z'] =  grid_points_df["z"]
            new_array ['mapped_x'] =  grid_points_df["x"]
            new_array ['mapped_y'] =  grid_points_df["y"]
            new_array ['mapped_z'] =  grid_points_df["z"]

        print(len(np.unique(transformed_points["id"])))


        # manualでつくったmaskでmaskしてみる。

        dt_mask = np.dtype([
            ('X(um)', 'f4'), ('Y(um)', 'f4'), ('Z(um)', 'f4'), ('atlasID', 'u2')
        ])

        mask_a =  np.fromfile("/export2/Imaging/ds4_Data4/atlas_R_test/CUBIC-space-grid-10um-resolution-transformed_with_annotations_modified_integrated_rounded_final.bin",  dtype=dt_mask)
        mask_a =  pd.DataFrame(mask_a)
        mask_a["x"] = mask_a["X(um)"] * (1/50)
        mask_a["y"] = mask_a["Y(um)"] * (1/50)
        mask_a["z"] = mask_a["Z(um)"] * (1/50)

        print(len(mask_a))
        end_time = datetime.datetime.now()
        print("Mask Read End time:", end_time)
        print("Duration:", end_time - start_time)

        import numpy as np
        import datetime
        from scipy.spatial import cKDTree

        def split_dataframe(df, n_splits=10):
            """DataFrameをn_splits等分に分割する"""
            chunk_size = len(df) // n_splits
            return [df[i:i + chunk_size] for i in range(0, len(df), chunk_size)]

        start_time = datetime.datetime.now()
        print("Start time:", start_time)

        # maskを10等分に分割
        splits = split_dataframe(mask_a[['x', 'y', 'z']], n_splits=10)

        # grid_points_dfの座標
        grid_points = grid_points_df[['x', 'y', 'z']].values

        # 全ての分割での検索結果を格納する配列
        all_distances = np.full(grid_points.shape[0], np.inf)
        all_indexes = np.full(grid_points.shape[0], -1)

        cou = 0
        for split in splits:
            print(cou)
            # 分割ごとにcKDTreeオブジェクトを作成
            tree = cKDTree(split.values)
            # queryを実行
            distances, indexes = tree.query(grid_points, k=1, distance_upper_bound=3, n_jobs=-1)

            # 距離がより短い場合は、その結果で更新
            better_mask = distances < all_distances
            all_distances[better_mask] = distances[better_mask]
            all_indexes[better_mask] = indexes[better_mask]
            cou += 1

            del tree, distances, indexes, better_mask
            gc.collect()

            end_time = datetime.datetime.now()
            print(str(cou) + "_End time:", end_time)
            print("Duration:", end_time - start_time)

        # 最短距離が3の平方根以内の点のみを抽出
        mask_manual = all_distances <= (3 ** 0.5) * (10/25) # 要検討。25 um scaleの √3なので。→ 10 um scaleの√3にした。

        del all_distances, all_indexes, mask_a
        gc.collect()

        print(np.sum(mask_manual))

        # mask_manualは最後に使って、annotation = 0を代入するのに使おうか。

        end_time = datetime.datetime.now()
        print("All End time:", end_time)
        print("Duration:", end_time - start_time)

        print("###############  SCA registration   #####################")

        import gc
        start_time = datetime.datetime.now()
        print("Start time:", start_time)

        mask1 = tifffile.imread('/export2/Imaging/ds4_Data4/atlas_R_test/cortex_grid.tif')
        mask1 [mask1 > 0] = 1.0 

        # 1の値を持つvoxelの座標を抽出
        coordinates4 = np.argwhere(mask1 == 1)

        # DataFrameに変換
        coordinates_df4 = pd.DataFrame(coordinates4, columns=['z', 'y', 'x'])

        mask3 = tifffile.imread('/export2/Imaging/ds4_Data4/atlas_R_test/shono_grid.tif')
        mask3 [mask3 > 0] = 1.0 

        # 1の値を持つvoxelの座標を抽出
        coordinates5 = np.argwhere(mask3 == 1)

        # DataFrameに変換
        coordinates_df5 = pd.DataFrame(coordinates5, columns=['z', 'y', 'x'])

        from scipy.spatial import cKDTree

        transformed_points = transformed_points.reset_index(drop=True)

        # cKDTreeオブジェクトを作成
        tree = cKDTree(coordinates_df4[['x', 'y', 'z']].values)

        # result_3D_points2_dfの各点に対して、coordinates_dfの点への最短距離を計算
        #result_3D_points3_extracted = result_3D_points3[(result_3D_points3["y"] > 395 - 2)& (result_3D_points3["y"] < 425 + 2) ]
        distances, indexes = tree.query(transformed_points[['x', 'y', 'z']].values, k=1, distance_upper_bound = 3, n_jobs = -1)

        # 最短距離が100μm以内の点のみを抽出
        mask_cortex = distances <=  1.4143
        mask_1 = transformed_points["y"] < 420
        #transformed_points_cortex = transformed_points[(mask) | (~mask & mask2)]

        #transformed_points_cortex

        from scipy.spatial import cKDTree

        transformed_points = transformed_points.reset_index(drop=True)

        # cKDTreeオブジェクトを作成
        tree = cKDTree(coordinates_df5[['x', 'y', 'z']].values)
        #transformed_points
        # result_3D_points2_dfの各点に対して、coordinates_dfの点への最短距離を計算
        #result_3D_points3_extracted = result_3D_points3[(result_3D_points3["y"] > 395 - 2)& (result_3D_points3["y"] < 425 + 2) ]
        distances, indexes = tree.query(transformed_points[['x', 'y', 'z']].values, k=1, distance_upper_bound = 3, n_jobs = -1)

        # 最短距離が100μm以内の点のみを抽出
        mask_shono = distances <= 1.4143
        mask_2 = transformed_points["y"] >= 420 
        #transformed_points_nokan_shono= transformed_points[(mask) | (~mask & mask1 & mask2)]

        transformed_points_cortex = transformed_points[(mask_cortex) | (~mask_cortex & ~mask_shono & mask_1)]
        transformed_points_nokan_shono = transformed_points[(mask_shono) | (~mask_cortex & ~mask_shono & mask_2)]
        transformed_points_cortex_only = transformed_points[(mask_cortex & ~mask_shono) | (~mask_cortex & ~mask_shono & mask_1)]
        transformed_points_shono_only = transformed_points[(~mask_cortex & mask_shono) | (~mask_cortex & ~mask_shono & mask_2)]

        del distances, indexes,mask1, mask3, mask_cortex, mask_shono, mask_1, mask_2, coordinates5, coordinates_df5, tree, coordinates4, coordinates_df4
        gc.collect()

        end_time = datetime.datetime.now()
        print("End time4:", end_time)
        print("Duration:", end_time - start_time)

        # 4つを連結していく
        # weightも



        import pandas as pd
        import numpy as np
        import os
        import re
        import ants
        import datetime
        from concurrent.futures import ThreadPoolExecutor, as_completed
        import gc

        print("######## weight calculate #############")

        def compute_weight(y, left_overlap_start, left_overlap_end, right_overlap_start, right_overlap_end, left_min_weight, right_min_weight):
            # 重みを全て1で初期化
            weight = np.ones_like(y)

            # 左オーバーラップのガウス重み計算
            left_indices = (y >= left_overlap_start) & (y <= left_overlap_end)
            if np.any(left_indices):
                left_sigma = (left_overlap_end - left_overlap_start) / 5
                left_center = left_overlap_end  # 中心を左オーバーラップの終端に設定
                weight[left_indices] = np.exp(-((y[left_indices] - left_center) ** 2) / (2 * left_sigma ** 2))
                # 左オーバーラップの重みをスケーリング
                weight[left_indices] = left_min_weight + (1 - left_min_weight) * (weight[left_indices] - np.min(weight[left_indices])) / (np.max(weight[left_indices]) - np.min(weight[left_indices]))

            # 右オーバーラップのガウス重み計算
            right_indices = (y >= right_overlap_start) & (y <= right_overlap_end)
            if np.any(right_indices):
                right_sigma = (right_overlap_end - right_overlap_start) / 5
                right_center = right_overlap_start  # 中心を右オーバーラップの開始点に設定
                weight[right_indices] = np.exp(-((y[right_indices] - right_center) ** 2) / (2 * right_sigma ** 2))
                # 右オーバーラップの重みをスケーリング
                weight[right_indices] = right_min_weight + (1 - right_min_weight) * (weight[right_indices] - np.min(weight[right_indices])) / (np.max(weight[right_indices]) - np.min(weight[right_indices]))

            return weight
        # 処理したいタスクをこの関数内に実装します。
        def process_slice(i, transformed_points1, path, slice_num, additional_cortex, additional_shono): 


            start_slice, end_slice = slice_num

            print("slice: " +str(i)+  ", start:" + str(start_slice) + ", end: " + str(end_slice))

            #filtered_indices3 = pd.Series([False] * len(transformed_points))  # 初期化

            if start_slice == 0 and end_slice != 528:  # startの端
                print("start edge: add minus y points")
                filtered_indices3 = transformed_points1["y"] < end_slice

            elif end_slice == 528 and start_slice != 0:  # endの端
                print("end edge: add over y points")
                filtered_indices3 = transformed_points1["y"] >= start_slice

            elif start_slice == 0 and end_slice == 528:  # 全域
                print("both edge: add over y points")
                filtered_indices3 = pd.Series([True] * len(transformed_points1))

            else:  # 通常の範囲
                filtered_indices3 = (transformed_points1["y"] >= start_slice) & (transformed_points1["y"] < end_slice)


            # 行と列のインデックスをDataFrameに変換。ここでは、あえて、yとzを交換している。
            extracted_points_df2 = []
            extracted_points_df2 = pd.DataFrame({'x': transformed_points1["x"][filtered_indices3], 
                                           'y':  transformed_points1["z"][filtered_indices3], 
                                           'z': transformed_points1["y"][filtered_indices3]-start_slice})


            temp_transformed_points2 = ants.apply_transforms_to_points(
            dim=3,
            points=extracted_points_df2,
            transformlist=[path + "slice_"+ str(start_slice) + "_"+ str(end_slice) +"_fwd.mat",
                           path + "slice_" + str(start_slice)+"_"+ str(end_slice) +"_inv.nii.gz"
                           ],
            whichtoinvert = [True, False]
            )

            del extracted_points_df2
            gc.collect()

            end_time = datetime.datetime.now()
            print("2nd transfer " + str(i) + "_end:", end_time)
            print("Duration:", end_time - start_time)

            temp_transformed_points2 = pd.DataFrame({'id': transformed_points1["id"][filtered_indices3],'ori_y': transformed_points1["y"][filtered_indices3], 'x': temp_transformed_points2["x"], 'y': temp_transformed_points2["z"] + start_slice, 'z': temp_transformed_points2["y"]})
            del filtered_indices3
            gc.collect()

            print("start " + str(i) + "_" + str(len(temp_transformed_points2)))

            alpha = 0.02
            #alpha2 = 0.01
            if i == 0:
                temp_transformed_points2['weight'] = compute_weight(temp_transformed_points2['y'], 
                                                                       0, 10, 100, 220, alpha, alpha)  
                temp_transformed_points2 = temp_transformed_points2[temp_transformed_points2 ["y"] < 220]

            if i == 1:
                temp_transformed_points2['weight'] = compute_weight(temp_transformed_points2['y'], 
                                                                       100, 220, 270, 350 , alpha, alpha)
                temp_transformed_points2 = temp_transformed_points2[(temp_transformed_points2["y"] >= 100)
                                                & (temp_transformed_points2["y"] < 350)]

            if i == 2:
                temp_transformed_points2['weight'] = compute_weight(temp_transformed_points2['y'], 
                                                                       270, 350, 360, 450, alpha,alpha) 
                temp_transformed_points2 = temp_transformed_points2[((temp_transformed_points2["y"] >= 270)
                                                & (temp_transformed_points2["y"] < 450))]

                #additional_cortex = additional_cortex[additional_cortex['ori_y'] >= 445]

                extracted_points_df2 = []
                extracted_points_df2 = pd.DataFrame({'x': additional_cortex["x"], 
                                               'y':  additional_cortex["z"], 
                                               'z': additional_cortex["y"] - start_slice})


                temp_additional_cortex2 = ants.apply_transforms_to_points(
                dim=3,
                points=extracted_points_df2,
                transformlist=[path + "slice_"+ str(start_slice) + "_"+ str(end_slice) +"_fwd.mat",
                               path + "slice_" + str(start_slice)+"_"+ str(end_slice) +"_inv.nii.gz"
                               ],
                whichtoinvert = [True, False]
                )

                temp_additional_cortex2 = pd.DataFrame({'id': additional_cortex["id"],'ori_y': additional_cortex["y"], 'x': temp_additional_cortex2["x"], 'y': temp_additional_cortex2["z"] + start_slice, 'z': temp_additional_cortex2["y"]})

                temp_additional_cortex2 = temp_additional_cortex2[temp_additional_cortex2['ori_y']>= 444]

                temp_additional_cortex2['weight'] = np.ones_like(len(temp_additional_cortex2))
                temp_transformed_points2 = pd.concat([temp_transformed_points2, temp_additional_cortex2], ignore_index=True)

                del temp_additional_cortex2
                gc.collect()

            if i == 3:
                temp_transformed_points2['weight'] = compute_weight(temp_transformed_points2['y'], 
                                                                      360, 450, 518, 528, alpha, alpha)
                temp_transformed_points2 = temp_transformed_points2[(temp_transformed_points2["y"] >= 360)]

                #additional_shono = additional_shono[additional_shono['ori_y'] < 387]

                extracted_points_df2 = []
                extracted_points_df2 = pd.DataFrame({'x': additional_shono["x"], 
                                               'y':  additional_shono["z"], 
                                               'z': additional_shono["y"] - start_slice})


                temp_additional_shono2 = ants.apply_transforms_to_points(
                dim=3,
                points=extracted_points_df2,
                transformlist=[path + "slice_"+ str(start_slice) + "_"+ str(end_slice) +"_fwd.mat",
                               path + "slice_" + str(start_slice)+"_"+ str(end_slice) +"_inv.nii.gz"
                               ],
                whichtoinvert = [True, False]
                )

                temp_additional_shono2 = pd.DataFrame({'id': additional_shono["id"],'ori_y': additional_shono["y"], 'x': temp_additional_shono2["x"], 'y': temp_additional_shono2["z"] + start_slice, 'z': temp_additional_shono2["y"]})

                temp_additional_shono2= temp_additional_shono2[temp_additional_shono2['ori_y'] < 387]

                temp_additional_shono2['weight'] = np.ones_like(len(temp_additional_shono2))
                temp_transformed_points2 = pd.concat([temp_transformed_points2, temp_additional_shono2], ignore_index=True)

                del temp_additional_shono2
                gc.collect()



            print("end " + str(i) + "_" +str(len(temp_transformed_points2)))

            return temp_transformed_points2


        start_time = datetime.datetime.now()
        print("Start time:", start_time)

        # 4つのregistrationの src点群とregistration方程式のpathを与える

        transformed_points_cortex = transformed_points_cortex.reset_index(drop=True)
        transformed_points_nokan_shono = transformed_points_nokan_shono.reset_index(drop=True)
        transformed_points_cortex_only = transformed_points_cortex_only.reset_index(drop=True)
        transformed_points_shono_only = transformed_points_shono_only.reset_index(drop=True)

        srcs = [transformed_points_cortex, transformed_points_cortex, transformed_points_cortex,  transformed_points_nokan_shono]
        additional_cortex = transformed_points_cortex_only
        additional_shono = transformed_points_shono_only
        main_path = "/export2/Imaging/ds4_Data4/atlas_R_test/"
        paths = [main_path + "3D_block_registerded_cortex_inv_fwd/", main_path + "3D_block_registerded_cortex_inv_fwd/", main_path + "3D_block_registerded_cortex_inv_fwd/",
               main_path + "3D_block_registerded_nokan_shono_inv_fwd/"]

        slice_nums = [(0, 250), (0, 528), (250, 528), (250, 528)]

        # 結果を格納する一時リストを用意します。
        results = []

        df = pd.DataFrame()  # 処理結果を格納する一時リスト

        # ThreadPoolExecutorを使って並列処理を行います。
        with ThreadPoolExecutor(max_workers=4) as executor:
            futures4 = {executor.submit(process_slice, i,  srcs[i], paths[i], slice_nums[i], additional_cortex, additional_shono): i for i in range(len(srcs))}

            for future in as_completed(futures4):
                index = futures4[future]
                try:
                    result = future.result()
                    results.append((index, result))  # インデックスと結果をタプルとして保存
                except Exception as e:
                    print(f"Error processing slice: {index} with error {e}")

        # インデックスでソートしてからdfに結果を結合します。
        # こうすることで、結果がiの順番通りになることを保証します。

        if 1== 1: 
            results.sort(key=lambda x: x[0])  # インデックスで結果をソート
            for _, result in results:
                df = pd.concat([df, result], ignore_index=True)

            # 不要な変数の削除とガベージコレクションの実行
            del results, futures4
            gc.collect()

        end_time = datetime.datetime.now()
        print("End time:", end_time)
        print("Duration:", end_time - start_time)

        print("######## integrate #############")
        # integrate

        start_time = datetime.datetime.now()
        print("Start time:", start_time)

        # 各idのweightの合計を計算
        weight_sum = df.groupby('id')['weight'].transform('sum')

        end_time = datetime.datetime.now()
        print("End time1:", end_time)
        print("Duration:", end_time - start_time)

        # weightを標準化
        df['normalized_weight'] = df['weight'] / weight_sum

        del weight_sum
        gc.collect()

        end_time = datetime.datetime.now()
        print("End time2:", end_time)
        print("Duration:", end_time - start_time)

        # weightとx, y, zを掛け合わせて加重値を計算
        df['x'] = df['normalized_weight'] * df['x']
        df['y'] = df['normalized_weight'] * df['y']
        df['z'] = df['normalized_weight'] * df['z']

        end_time = datetime.datetime.now()
        print("End time3:", end_time)
        print("Duration:", end_time - start_time)

        # 各idでx, y, zの和を計算
        averaged_coordinates = df.groupby('id').agg(
            x=('x', 'sum'),
            y=('y', 'sum'),
            z=('z', 'sum')
        ).reset_index()

        end_time = datetime.datetime.now()
        print("End time4:", end_time)
        print("Duration:", end_time - start_time)

        # ここで追加のAffineをするか

        src_averaged_coordinates = averaged_coordinates.copy()

        averaged_coordinates = ants.apply_transforms_to_points(
            dim=3,
            points=src_averaged_coordinates,
            transformlist=["/export2/Imaging/ds4_Data4/atlas_R_test/3D_block_registerded_cortex_SCA_for_SCN_registration_ds4_inv_fwd/slice_0_528_inv_Affine_horizontal.mat"],
            whichtoinvert = [True]
        )

        del src_averaged_coordinates
        gc.collect()

        end_time = datetime.datetime.now()
        print("Affine End time4.5:", end_time)
        print("Duration:", end_time - start_time)

        if 1==0:
            scale = 1/50

            depth_ori = 320 # 320
            #height_ori =int(275* scale * 80)#int(np.floor(np.max(df2["Y(um)"]*scale))) + 20
            height_ori = 528
            width_ori = 456

            print(depth_ori)
            print(height_ori)
            print(width_ori)

            img_filename_Nuclear_Isocortex_to_ori = "/export2/Imaging/ds4_Data4/atlas_R_test/temp2_.tif"

            img_N_ori,_ = np.histogramdd(
                np.vstack([
                    averaged_coordinates["z"],
                    averaged_coordinates["y"], 
                    averaged_coordinates["x"] 
                    ]).T,
                    bins=(depth_ori, height_ori, width_ori),
                    range=[(0,depth_ori),(0,height_ori),(0,width_ori)]
                )

            tifffile.imsave(
                    img_filename_Nuclear_Isocortex_to_ori,
                    img_N_ori.astype(np.float32)
                )

            del img_N_ori
            gc.collect()

        end_time = datetime.datetime.now()
        print("End time5:", end_time)
        print("Duration:", end_time - start_time)


        print(len(np.unique(df["id"])))
        print(len(np.unique(transformed_points["id"])))

        print("##############  annotation ############")

        ## atlas annotationをつける用のコード

        start_time = datetime.datetime.now()
        print("Annotation Start time:", start_time)

        annotation_10 = tifffile.imread('/export2/Imaging/ds4_Data4/atlas/annotation_10_LUT_modified_final.tif') # 545 → 869
        #annotation_25 = tifffile.imread('/export2/Imaging/ds4_Data4/atlas/annotation_25_LUT_modified.tif')

        # 0でない値が入っている座標とその値を取得
        non_zero_coords = np.argwhere(annotation_10)
        non_zero_values = annotation_10[non_zero_coords[:, 0], non_zero_coords[:, 1], non_zero_coords[:, 2]]

        # DataFrameに変換
        df_non_zero = pd.DataFrame(non_zero_coords, columns=['z', 'y', 'x'])
        df_non_zero['id'] = np.round(non_zero_values)

        df_non_zero['z'] = (np.round(df_non_zero['z']) + 0.5) * (10/25) # 10のまま.+ 0.5しないと、voxelの中心にないので。
        df_non_zero['y'] = (np.round(df_non_zero['y']) + 0.5) * (10/25)
        df_non_zero['x'] = (np.round(df_non_zero['x']) + 0.5) * (10/25)

        del  non_zero_coords, non_zero_values
        gc.collect()

        end_time = datetime.datetime.now()
        print("End time1:", end_time)
        print("Duration:", end_time - start_time)


        # cKDTreeオブジェクトを作成

        start_time = datetime.datetime.now()
        print("Tree Start time:", start_time)


        def split_dataframe(df, n_splits=10):
            """DataFrameをn_splits等分に分割する"""
            chunk_size = len(df) // n_splits
            return [df.iloc[i:i + chunk_size] for i in range(0, len(df), chunk_size)]


        # df_non_zeroを10等分に分割
        splits = split_dataframe(df_non_zero[['x', 'y', 'z']], n_splits=10)

        # averaged_coordinatesの座標
        averaged_coords = averaged_coordinates[['x', 'y', 'z']].values

        # 全ての分割での検索結果を格納するための配列
        all_distances = np.full(averaged_coords.shape[0], np.inf)
        all_indexes = np.full(averaged_coords.shape[0], -1)

        cou = 0
        for split in splits:
            # 分割ごとにcKDTreeオブジェクトを作成
            print(cou)
            tree = cKDTree(split.values)
            distances, indexes = tree.query(averaged_coords, k=1, distance_upper_bound=2, n_jobs=-1)
            # 距離が以前の分割の結果よりも短い場合は更新
            mask = distances < all_distances
            all_distances[mask] = distances[mask]
            all_indexes[mask] = indexes[mask] + split.index.start  # インデックスのオフセットを追加
            cou += 1
            del tree, distances, indexes, mask
            gc.collect()

            end_time = datetime.datetime.now()
            print(str(cou) + " End time:", end_time)
            print("Duration:", end_time - start_time)

        # 距離が特定の閾値以下の場合はnearest_idを更新
        distance_threshold = (3 ** 0.5) * (10 / 25)  # √2  25 um scaleの√2  → 10 um scaleの√3
        mask = all_distances <= distance_threshold

        # averaged_coordinatesにnearest_id列を追加し、df_non_zeroのidで更新
        averaged_coordinates['nearest_id'] = 0  # 0で初期化
        averaged_coordinates.loc[mask, 'nearest_id'] = df_non_zero.iloc[all_indexes[mask]]['id'].values


        # 追加のmask: annotation_10の範囲内だけに限るためのmask。外側がはみ出しがちだから。
        # annotation_10の形状を取得
        depth, height, width = annotation_10.shape

        # df_saveのz, y, x座標を整数に丸める
        z_indices = (averaged_coordinates['z']*(25/10)).astype(int)
        y_indices = (averaged_coordinates['y']*(25/10)).astype(int)
        x_indices = (averaged_coordinates['x']*(25/10)).astype(int)

        # 座標がannotation_10の範囲内にあるかどうかを確認
        in_range_mask = (z_indices >= 0) & (z_indices < depth) & (y_indices >= 0) & (y_indices < height) & (x_indices >= 0) & (x_indices < width)

        # 範囲内のインデックスでannotation_10を参照するためのマスクを用いて、値を抽出
        extracted_values = annotation_10[z_indices[in_range_mask], y_indices[in_range_mask], x_indices[in_range_mask]]

        # 抽出した値が0より大きい要素にTrueが設定される新しいマスクを作成
        valid_values_mask = np.zeros(len(averaged_coordinates), dtype=bool)
        valid_values_mask[in_range_mask] = extracted_values > 0

        # 0より大きい要素に対するブールマスクを作成
        #annotation_mask = annotation_10 > 0

        # 範囲内でのみフィルタリング条件を適用
        # インデックスエラーを避けるために、まず範囲内であることを確認
        #filtered_mask = in_range_mask & annotation_mask[z_indices, y_indices, x_indices]

        # 条件を満たす行のみを含むDataFrameを作成
        averaged_coordinates['nearest_id'][~valid_values_mask] = 0

        del annotation_10, z_indices, y_indices, x_indices, in_range_mask, valid_values_mask, extracted_values
        gc.collect()

        # mask_manual

        #dt_annotation2 = np.dtype([
        #('stitched_x', 'f4'), ('stitched_y', 'f4'), ('stitched_z', 'f4'),
        #('mapped_x', 'f4'), ('mapped_y', 'f4'), ('mapped_z', 'f4'),
        #('SCA_x', 'f4'), ('SCA_y', 'f4'), ('SCA_z', 'f4'),
        #('atlas_id', 'u2')
        #])

        #dt_annotation2 = np.dtype([
        #('stitched_x', 'f4'), ('stitched_y', 'f4'), ('stitched_z', 'f4'),
        #('mapped_x', 'f4'), ('mapped_y', 'f4'), ('mapped_z', 'f4'),
        #('SCA_x', 'f4'), ('SCA_y', 'f4'), ('SCA_z', 'f4'),
        #('atlas_id', 'u2')
        #])

        new_array ['SCA_x'] = averaged_coordinates ["x"] * 25
        new_array ['SCA_y'] = averaged_coordinates ["y"] * 25
        new_array ['SCA_z'] = averaged_coordinates ["z"] * 25
        new_array ['atlas_id'][mask_manual] = averaged_coordinates ["nearest_id"][mask_manual]

        if grid_mode==0:
            new_array.tofile(fw_dir_nu + "/SCA_registered_cell_coordinates_ver2_2.bin")
        if grid_mode==1:
            #new_array.tofile(fw_dir_nu + "/SCA_registered_cell_coordinates_ver2_" + str(cou3) + ".bin")
            new_array.tofile(fw_dir_nu + "/SCA_registered_cell_coordinates_ver2_" + str(cou3) + ".bin")

        del all_distances, all_indexes, mask, averaged_coords, splits, new_array
        gc.collect()

        end_time = datetime.datetime.now()
        print("Averaged cell coordinates Save End time:", end_time)
        print("Duration:", end_time - start_time)

        print(averaged_coordinates["nearest_id"].max())


        #del averaged_coordinates_keys, df_non_zero_keys, key_id_map
        #gc.collect()
        # averaged_coordinatesにnearest_id列を追加し、df_non_zeroのidで更新


        # 結合結果から 'nearest_id' を更新
        # 一致する 'key' がある場合のみ 'id' を 'nearest_id' に代入
        #averaged_coordinates['nearest_id'] = merged_df['id'].fillna(0).astype(int)

        #averaged_coordinates.loc[mask, 'nearest_id'] = df_non_zero.iloc[all_indexes[mask]]['id'].values

        # mask_manual

        #dt_annotation2 = np.dtype([
        #('stitched_x', 'f4'), ('stitched_y', 'f4'), ('stitched_z', 'f4'),
        #('mapped_x', 'f4'), ('mapped_y', 'f4'), ('mapped_z', 'f4'),
        #('SCA_x', 'f4'), ('SCA_y', 'f4'), ('SCA_z', 'f4'),
        #('atlas_id', 'u2')
        #])


        # cKDTreeオブジェクトを作成

        start_time = datetime.datetime.now()
        print("Start time:", start_time)

        tree = []

        averaged_coordinates_part = averaged_coordinates [averaged_coordinates["nearest_id"]>0]
        tree = cKDTree(averaged_coordinates_part[['x', 'y', 'z']].values)

        end_time = datetime.datetime.now()
        print("End time1:", end_time)
        print("Duration:", end_time - start_time)

        # averaged_coordinatesの各点に対して、df_non_zeroの点への最短距離を計算
        distances2, indexes2 = tree.query(df_non_zero[['x', 'y', 'z']].values, k=1, distance_upper_bound=2, n_jobs=-1)

        del tree, averaged_coordinates_part
        gc.collect()



        end_time = datetime.datetime.now()
        print("All End time2", end_time)
        print("Duration:", end_time - start_time)


        print("#############  calculate region volume   ################")

        data_GO = pd.read_csv(filepath_or_buffer="/export3/Imaging/atlas_for_fig/GOvsID_6region_removed.csv", encoding="ms932", sep=",")
        # data_GO["id"]のユニークな値を取得
        ids = data_GO["id"].astype(int)
        ids=np.array(ids)


        mask = distances2 <= 2

        depth_ori = 800 
        height_ori = 1320
        width_ori = 1140

        annotation_10 = tifffile.imread('/export2/Imaging/ds4_Data4/atlas/annotation_10_LUT_modified_final.tif') # 545 → 869
        #annotation_25 = tifffile.imread('/export2/Imaging/ds4_Data4/atlas/annotation_25_LUT_modified.tif')

        # 0でない値が入っている座標とその値を取得
        non_zero_coords = np.argwhere(annotation_10)
        non_zero_values = annotation_10[non_zero_coords[:, 0], non_zero_coords[:, 1], non_zero_coords[:, 2]]

        # DataFrameに変換
        df_non_zero = pd.DataFrame(non_zero_coords, columns=['z', 'y', 'x'])
        df_non_zero['id'] = np.round(non_zero_values)

        df_non_zero['z'] = np.round(df_non_zero['z']) #+ 0.5 #* (10/25) # 10のまま.+ 0.5しないと、voxelの中心にないので。
        df_non_zero['y'] = np.round(df_non_zero['y']) #+ 0.5 #* (10/25)
        df_non_zero['x'] = np.round(df_non_zero['x']) #+ 0.5 #* (10/25)

        del annotation_10, non_zero_coords, non_zero_values
        gc.collect()

        # インデックスとIDの準備
        z_indices = df_non_zero['z'][mask].values.astype(int)
        y_indices = df_non_zero['y'][mask].values.astype(int)
        x_indices = df_non_zero['x'][mask].values.astype(int)
        annotation_10_ids = df_non_zero['id'][mask].values.astype(np.uint16)

        # 確認用：3D配列のデータ型をidsと一致させます
        image_3d = np.zeros((depth_ori, height_ori, width_ori), dtype=np.uint16)

        # IDの割り当て
        image_3d[z_indices, y_indices, x_indices] = annotation_10_ids

        # TIFFファイルとして保存
        if grid_mode==0:
            tifffile.imwrite(fw_dir_nu +  "/annotated_annotation_10_2.tif" , image_3d)
        if grid_mode==1:
            tifffile.imwrite(fw_dir_nu +  "/annotated_annotation_10_"+str(cou3)+".tif" , image_3d)

        del z_indices, y_indices, x_indices, annotation_10_ids, image_3d
        gc.collect()

        end_time = datetime.datetime.now()
        print("Tiff save End time1:", end_time)
        print("Duration:", end_time - start_time)

        # マスクを適用した後のid列から、各IDの出現回数を集計
        id_counts = df_non_zero['id'][mask].value_counts()

        # ユニークなIDリストに基づいて、集計結果をフィルタリング（存在しないIDは0として扱う）
        filtered_counts = id_counts.reindex(ids, fill_value=0)

        # DataFrameを作成
        df_filtered = filtered_counts.reset_index()
        df_filtered.columns = ['atlasID', 'iso_10um_voxel_count']  # 列名を設定

        # CSVに保存
        if grid_mode==0:
            csv_file_path =  fw_dir_nu +  "/Regional_iso_10um_voxel_count2.csv"  # 保存するファイルのパス
        if grid_mode==1:
            csv_file_path =  fw_dir_nu +  "/Regional_iso_10um_voxel_count_"+str(cou3)+".csv"  # 保存するファイルのパス
            #csv_file_path =  fw_dir_nu +  "/Regional_iso_10um_voxel_count.csv"  # 保存するファイルのパス
        df_filtered.to_csv(csv_file_path, index=False)

        del distances2, indexes2, df_non_zero, transformed_points, grid_points_df, averaged_coordinates, mask_manual
        gc.collect()

        cou3 += 1
    except Exception as e:
        print(f"Error loading data for {Sample}: {e}")

Start Start time: 2024-05-10 14:24:26.749544
### Read parameter file ###
/home/mitani/CUBIC-informatics/param/Neuronomics/8w_B6JWT_atlas_1_2022_0924_1847/param_multichannel-rank.json
/export3/Imaging/Axial/Neurorology/8w_B6JWT_atlas_1_2022_0924_1847/nu_R_/result/
FW reading
emp
FW reading end: 2024-05-10 14:25:09.887635
Duration: 0:00:43.138091
RV reading
emp
emp
emp
emp
emp
emp
emp
emp
emp
emp
READ OK
RV reading end: 2024-05-10 14:25:43.014947
Duration: 0:00:33.126929
parallel_filter end: 2024-05-10 14:26:27.670815
Duration: 0:00:44.654984
98023723
[( 9378.955, 21404.432 , 7173.6655,  9406.113, 21449.094 , 7204.216, 0., 0., 0., 0)
 ( 8865.565, 22016.936 , 7140.718 ,  8891.377, 22067.73  , 7171.132, 0., 0., 0., 0)
 ( 9119.072, 21572.7   , 7102.2515,  9145.629, 21619.504 , 7132.36 , 0., 0., 0., 0)
 ...
 (13869.83 ,  2765.7266, 8513.973 , 13911.063,  2771.9543, 8553.201, 0., 0., 0., 0)
 (13493.644,  3522.5898, 8513.363 , 13533.683,  3530.557 , 8552.5  , 0., 0., 0., 0)
 (13208.224,  2961.

105733370
105733370
##############  annotation ############
Annotation Start time: 2024-05-10 16:08:51.067691
End time1: 2024-05-10 16:10:06.229931
Duration: 0:01:15.162240
Tree Start time: 2024-05-10 16:10:06.231273
0
1 End time: 2024-05-10 16:11:20.578903
Duration: 0:01:14.347630
1
2 End time: 2024-05-10 16:12:02.282070
Duration: 0:01:56.050797
2
3 End time: 2024-05-10 16:12:43.271284
Duration: 0:02:37.040011
3
4 End time: 2024-05-10 16:13:29.584410
Duration: 0:03:23.353137
4
5 End time: 2024-05-10 16:14:10.860327
Duration: 0:04:04.629054
5
6 End time: 2024-05-10 16:14:53.923592
Duration: 0:04:47.692319
6
7 End time: 2024-05-10 16:15:37.532688
Duration: 0:05:31.301415
7
8 End time: 2024-05-10 16:16:18.654486
Duration: 0:06:12.423213
8
9 End time: 2024-05-10 16:17:00.624727
Duration: 0:06:54.393454
9
10 End time: 2024-05-10 16:17:41.689789
Duration: 0:07:35.458516
Averaged cell coordinates Save End time: 2024-05-10 16:18:22.029319
Duration: 0:08:15.798046
61277
Start time: 2024-05-10 

End time4: 2024-05-10 17:30:10.035869
Duration: 0:01:31.452640
######## weight calculate #############
Start time: 2024-05-10 17:30:10.036065
slice: 0, start:0, end: 250
start edge: add minus y points
slice: 1, start:0, end: 528
slice: 2, start:250, end: 528both edge: add over y points

end edge: add over y points
slice: 3, start:250, end: 528
end edge: add over y points
2nd trabsfer 0_end: 2024-05-10 17:31:04.751737
Duration: 0:00:54.715672
2nd trabsfer 3_end: 2024-05-10 17:31:05.662630
Duration: 0:00:55.626565
2nd trabsfer 2_end: 2024-05-10 17:31:06.639446
Duration: 0:00:56.603381
start 0_33364505
2nd trabsfer 1_end: 2024-05-10 17:31:37.435953
Duration: 0:01:27.399888
start 3_36072558
start 2_40253848
end 0_27109119
start 1_73618353
end 3_36064805
end 1_46923904
end 2_31757522
End time: 2024-05-10 17:32:38.278359
Duration: 0:02:28.242294
######## integrate #############
Start time: 2024-05-10 17:32:38.278527
End time1: 2024-05-10 17:33:21.558234
Duration: 0:00:43.279707
End time2: 20

Trasform Start time: 2024-05-10 18:40:27.211220
Trasform End time: 2024-05-10 18:40:43.090200
Duration: 0:00:15.878980
99449746
1113543350
Mask Read End time: 2024-05-10 18:42:54.143205
Duration: 0:02:26.931985
Start time: 2024-05-10 18:42:54.143438
0
1_End time: 2024-05-10 18:45:28.813750
Duration: 0:02:34.670312
1
2_End time: 2024-05-10 18:46:59.472379
Duration: 0:04:05.328941
2
3_End time: 2024-05-10 18:48:28.313031
Duration: 0:05:34.169593
3
4_End time: 2024-05-10 18:49:57.187816
Duration: 0:07:03.044378
4
5_End time: 2024-05-10 18:51:27.120263
Duration: 0:08:32.976825
5
6_End time: 2024-05-10 18:52:56.523871
Duration: 0:10:02.380433
6
7_End time: 2024-05-10 18:54:25.377517
Duration: 0:11:31.234079
7
8_End time: 2024-05-10 18:55:53.472051
Duration: 0:12:59.328613
8
9_End time: 2024-05-10 18:57:20.956053
Duration: 0:14:26.812615
9
10_End time: 2024-05-10 18:58:48.509886
Duration: 0:15:54.366448
97272571
All End time: 2024-05-10 18:58:52.988513
Duration: 0:15:58.845075
##############

All End time2 2024-05-10 20:01:44.992794
Duration: 0:02:06.201821
#############  calculate region volume   ################
Tiff save End time1: 2024-05-10 20:03:58.153801
Duration: 0:04:19.362828
Start Start time: 2024-05-10 20:04:12.049970
### Read parameter file ###
/home/mitani/CUBIC-informatics/param/Neuronomics/#4_APPmodel_Ctr3m_4_2023_0427_1444/param_multichannel-rank.json
/export3/Imaging/Axial/Neurorology/#4_APPmodel_Ctr3m_4_2023_0427_1444/nu_R_/result/
FW reading
empempempemp



emp
emp
emp
emp
emp
FW reading end: 2024-05-10 20:05:02.196592
Duration: 0:00:50.146622
RV reading
emp
emp
emp
emp
READ OK
RV reading end: 2024-05-10 20:05:59.122315
Duration: 0:00:56.925367
parallel_filter end: 2024-05-10 20:09:01.208527
Duration: 0:03:02.085956
104047017
[( 7715.447 , 23440.762, 7288.7363,  6119.6226, 19815.945 , 5616.1763, 0., 0., 0., 0)
 ( 8166.706 , 23393.953, 7289.2896,  6534.3677, 19797.127 , 5605.799 , 0., 0., 0., 0)
 ( 8210.262 , 23436.154, 7289.602 ,  6580.3296, 19851.861 , 

End time3: 2024-05-10 21:17:27.751624
Duration: 0:00:48.024611
End time4: 2024-05-10 21:19:03.021449
Duration: 0:02:23.294436
Affine End time4.5: 2024-05-10 21:19:27.651040
Duration: 0:02:47.924027
End time5: 2024-05-10 21:19:27.651194
Duration: 0:02:47.924181
101922677
101922677
##############  annotation ############
Annotation Start time: 2024-05-10 21:19:36.703190
End time1: 2024-05-10 21:21:02.260476
Duration: 0:01:25.557286
Tree Start time: 2024-05-10 21:21:02.260641
0
1 End time: 2024-05-10 21:22:17.297210
Duration: 0:01:15.036569
1
2 End time: 2024-05-10 21:22:58.541576
Duration: 0:01:56.280935
2
3 End time: 2024-05-10 21:23:40.172143
Duration: 0:02:37.911502
3
4 End time: 2024-05-10 21:24:18.057712
Duration: 0:03:15.797071
4
5 End time: 2024-05-10 21:24:59.140674
Duration: 0:03:56.880033
5
6 End time: 2024-05-10 21:25:40.263721
Duration: 0:04:38.003080
6
7 End time: 2024-05-10 21:26:21.064894
Duration: 0:05:18.804253
7
8 End time: 2024-05-10 21:26:59.337656
Duration: 0:05:57.0

10_End time: 2024-05-10 22:40:59.502045
Duration: 0:16:12.457579
93115515
All End time: 2024-05-10 22:41:02.845625
Duration: 0:16:15.801159
###############  SCA registration   #####################
Start time: 2024-05-10 22:41:02.845768
End time4: 2024-05-10 22:42:36.507799
Duration: 0:01:33.662031
######## weight calculate #############
Start time: 2024-05-10 22:42:36.507998
slice: 0, start:0, end: 250slice: 1, start:0, end: 528slice: 2, start:250, end: 528
end edge: add over y points

slice: 3, start:250, end: 528both edge: add over y points

end edge: add over y points

start edge: add minus y points
2nd trabsfer 0_end: 2024-05-10 22:43:33.298047
Duration: 0:00:56.790049
2nd trabsfer 3_end: 2024-05-10 22:43:34.152774
Duration: 0:00:57.644776
2nd trabsfer 2_end: 2024-05-10 22:43:34.903815
Duration: 0:00:58.395817
start 0_32176679
start 3_36244289
start 2_39430721
2nd trabsfer 1_end: 2024-05-10 22:44:05.266681
Duration: 0:01:28.758683
end 0_25744162
start 1_71607400
end 3_36221632
end

Trasform Start time: 2024-05-10 23:50:39.004303
Trasform End time: 2024-05-10 23:50:53.854889
Duration: 0:00:14.850586
95610827
1113543350
Mask Read End time: 2024-05-10 23:53:03.788086
Duration: 0:02:24.783783
Start time: 2024-05-10 23:53:03.788285
0
1_End time: 2024-05-10 23:55:38.428839
Duration: 0:02:34.640554
1
2_End time: 2024-05-10 23:57:09.480386
Duration: 0:04:05.692101
2
3_End time: 2024-05-10 23:58:38.178271
Duration: 0:05:34.389986
3
4_End time: 2024-05-11 00:00:07.197362
Duration: 0:07:03.409077
4
5_End time: 2024-05-11 00:01:36.299240
Duration: 0:08:32.510955
5
6_End time: 2024-05-11 00:03:05.893654
Duration: 0:10:02.105369
6
7_End time: 2024-05-11 00:04:34.936287
Duration: 0:11:31.148002
7
8_End time: 2024-05-11 00:06:04.835496
Duration: 0:13:01.047211
8
9_End time: 2024-05-11 00:07:37.453932
Duration: 0:14:33.665647
9
10_End time: 2024-05-11 00:09:07.950879
Duration: 0:16:04.162594
93514282
All End time: 2024-05-11 00:09:11.140071
Duration: 0:16:07.351786
##############

All End time2 2024-05-11 01:10:19.262368
Duration: 0:02:02.221812
#############  calculate region volume   ################
Tiff save End time1: 2024-05-11 01:12:16.777911
Duration: 0:03:59.737355
Start Start time: 2024-05-11 01:12:30.601693
### Read parameter file ###
/home/mitani/CUBIC-informatics/param/Neuronomics/56w_B6JWT_atlas_1_2022_1128_1548/param_multichannel-rank.json
/export3/Imaging/Axial/Neurorology/56w_B6JWT_atlas_1_2022_1128_1548/nu_R_/result/
FW reading
emp
emp
emp
empemp

emp
emp
FW reading end: 2024-05-11 01:13:16.988157
Duration: 0:00:46.386464
RV reading
READ OK
RV reading end: 2024-05-11 01:14:09.564485
Duration: 0:00:52.575949
parallel_filter end: 2024-05-11 01:17:09.238808
Duration: 0:02:59.674075
101439980
[( 6541.325 , 23438.902 ,  7137.0674,  5007.315, 19664.19  , 5711.601 , 0., 0., 0., 0)
 ( 7208.892 , 23375.662 ,  6979.1245,  5618.6  , 19601.947 , 5579.615 , 0., 0., 0., 0)
 ( 7023.9395, 23542.105 ,  6971.1655,  5448.691, 19768.99  , 5563.1597, 0., 0., 0., 0)

End time4: 2024-05-11 02:24:54.098314
Duration: 0:02:22.393027
Affine End time4.5: 2024-05-11 02:25:18.416538
Duration: 0:02:46.711251
End time5: 2024-05-11 02:25:18.416677
Duration: 0:02:46.711390
99930301
99930301
##############  annotation ############
Annotation Start time: 2024-05-11 02:25:26.928361
End time1: 2024-05-11 02:26:38.572857
Duration: 0:01:11.644496
Tree Start time: 2024-05-11 02:26:38.573025
0
1 End time: 2024-05-11 02:27:49.385028
Duration: 0:01:10.812003
1
2 End time: 2024-05-11 02:28:30.495661
Duration: 0:01:51.922636
2
3 End time: 2024-05-11 02:29:11.829679
Duration: 0:02:33.256654
3
4 End time: 2024-05-11 02:29:52.709488
Duration: 0:03:14.136463
4
5 End time: 2024-05-11 02:30:33.820187
Duration: 0:03:55.247162
5
6 End time: 2024-05-11 02:31:15.609115
Duration: 0:04:37.036090
6
7 End time: 2024-05-11 02:31:56.073226
Duration: 0:05:17.500201
7
8 End time: 2024-05-11 02:32:35.843667
Duration: 0:05:57.270642
8
9 End time: 2024-05-11 02:33:17.287623
Duration: 0:06:38.